# Simulated Annealing

We will be trying to find and visualize the path between Equestrian Statue of Edward VII and Bahen Center of Technology around Toronto University campus using simulated annealing

__SIMULATED-ANNEALING__(_source_ , _destination_ , _schedule_ , _num of iterations_ ) __returns__ a route  
&emsp;_current_ &larr; random-route between _source_ and _destination_  
&emsp;__for__ _num of iterations_ __do__    
&emsp;&emsp; _T_ &larr; _schedule(t)_  
&emsp;&emsp; _neighbors_ &larr; _current_.NEIGHBOURS  
&emsp;&emsp; _next_ &larr; randomly choose one state from _neighbors_  
&emsp;&emsp; _$\Delta$_ _E_ &larr; _next_._COST_ - _current_._COST_  
&emsp;&emsp; __if__ _$\Delta$_ _E_ &lt; 0 __then__ _current_ &larr; _next_    
&emsp;&emsp; __else__ _current_ &larr; _next_ only with probability of ${e}^{-\frac{\Delta E}{T}}$  
&emsp;__endfor__  
&emsp; _route_ &larr; _current_  
&emsp; __return__ _route_

In [ ]:
import osmnx as ox
import time, itertools
from tqdm import tqdm
from utilities import *

In [ ]:
location_point = (43.661667, -79.395)
G = ox.graph_from_point(location_point, dist=300, clean_periphery=True, simplify=True)
fig, ax = ox.plot_graph(G)

Here you need to specify which node from our graph is the source (Equestrian Statue of Edward VII) and which is the destination node (Bahen Center of Technology). You can do so by acquiring the decimal coordinates of the desired node and use [```osmnx.distance.get_nearest_node```](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.get_nearest_node) method

I used the aforementioned method and found that the nodes for destination and source are 389677909, 55808290 respectively

In [ ]:
highlighted = [389677909, 55808290]

# marking both the source and destination node

nc = ['r' if node in highlighted else '#336699' for node in G.nodes()]
ns = [50 if node in highlighted else 8 for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_size=ns, node_color=nc, node_zorder=2)

In [ ]:
draw_map(G, highlight = highlighted)

In [ ]:
%%capture
source(Node)

We need to generate a random paths from source and destination to be the starting state in simulated annealing. The following algorithm is just a variant of the typical graph search but instead of choosing the node to be expanded based on a certain policy like bfs/dfs, we just choose the node randomly from the frontier.

In [ ]:
source(randomized_search)

The default of the schedule function is that the initial temperature is 20 and it gets terminated after 100 unit time.

In [ ]:
source(exp_schedule)

In [ ]:
schedule = exp_schedule(200, 0.05, 10000)

This will take about 10 minutes to run, because generating children for a route is a bit computationally expensive.

In [ ]:
num_of_iterations = 100

In [ ]:
%%time
states = []
current = randomized_search(G, 55808290, 389677909)

# 
for t in tqdm(range(num_of_iterations)):
    T = schedule(t)
    states.append(cost(G, current))
    
    # generate 5 more paths to choose from
    neighbors = [*itertools.islice(children_route(G, current), 100)]
    next_choice = random.choice(neighbors)
    
    delta_e = cost(G, next_choice) - cost(G, current)  
    if delta_e < 0 or probability(np.exp(-1 * delta_e/T)):
        current = next_choice
        
    print(cost(G, current))
route = current

In [ ]:
cost(G, route)

Remember that the optimal cost for the route is 801.4639999999999  

You should get 830-ish as the cost of the route between source and destination without any tuning of the schedule function.

In [ ]:
fig, ax = ox.plot_graph_route(G, current)

In [ ]:
draw_route(G, route)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel("# iterations")
plt.ylabel("cost (meters)")
plt.plot(states)
plt.show()